In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Dropout

In [3]:
class Linear(Layer):
    def __init__(self):
        super(Linear, self).__init__()
        self.out_layer = Dense(1, activation=None)

    def call(self, inputs, **kwargs):
        output = self.out_layer(inputs)
        return output

class Dense_layer(Layer):
    def __init__(self, hidden_units, out_dim=1, activation='relu', dropout=0.0):
        super(Dense_layer, self).__init__()
        self.hidden_layers = [Dense(i, activation=activation) for i in hidden_units]
        self.out_layer = Dense(out_dim, activation=None)
        self.dropout = Dropout(dropout)

    def call(self, inputs, **kwargs):
        # inputs: [None, n*k]
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.dropout(x)
        output = self.out_layer(x)
        return output

class CIN(Layer):
    def __init__(self, cin_size):
        super(CIN, self).__init__()
        self.cin_size = cin_size  # 每层的矩阵个数

    def build(self, input_shape):
        # input_shape: [[None, n, k]None, n, k]
        self.field_num = [input_shape[1]] + self.cin_size # 每层的矩阵个数(包括第0层)

        self.cin_W = [self.add_weight(
                         name='w'+str(i),
                         shape=(1, self.field_num[0]*self.field_num[i], self.field_num[i+1]),
                         initializer=tf.initializers.glorot_uniform(),
                         regularizer=tf.keras.regularizers.l1_l2(1e-5),
                         trainable=True)
                      for i in range(len(self.field_num)-1)]

    def call(self, inputs, **kwargs):
        # inputs: [None, n, k], 只有SparseFeature 
        k = inputs.shape[-1]
        res_list = [inputs]
        X0 = tf.split(inputs, k, axis=-1)           # 最后维切成k份，list: k * [None, field_num[0], 1]
        for i, size in enumerate(self.field_num[1:]):
            Xi = tf.split(res_list[-1], k, axis=-1) # list: k * [None, field_num[i], 1]
            #A batch matrix multiplication with batch shape [2]:
            #高纬下也只是取倒数第二维度
            x = tf.matmul(X0, Xi, transpose_b=True) # list: k * [None, field_num[0], field_num[i]]
            x = tf.reshape(x, shape=[k, -1, self.field_num[0]*self.field_num[i]])
                                                    # [k, None, field_num[0]*field_num[i]]
            x = tf.transpose(x, [1, 0, 2])          # [None, k, field_num[0]*field_num[i]]
            x = tf.nn.conv1d(input=x, filters=self.cin_W[i], stride=1, padding='VALID')
                                                    # (None, k, field_num[i+1])
            x = tf.transpose(x, [0, 2, 1])          # (None, field_num[i+1], k)
            res_list.append(x)

        res_list = res_list[1:]   # 去掉X0
        res = tf.concat(res_list, axis=1)  # (None, field_num[1]+...+field_num[n], k)
        output = tf.reduce_sum(res, axis=-1)  # (None, field_num[1]+...+field_num[n])
        return output


In [6]:

import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding

class xDeepFM(Model):
    def __init__(self, feature_columns, cin_size, hidden_units, out_dim=1, activation='relu', dropout=0.0):
        super(xDeepFM, self).__init__()
        self.dense_feature_columns, self.sparse_feature_columns = feature_columns
        self.embed_layers = [Embedding(feat['feat_onehot_dim'], feat['embed_dim'])
                                    for feat in self.sparse_feature_columns]
        self.linear = Linear()
        self.dense_layer = Dense_layer(hidden_units, out_dim, activation, dropout)
        self.cin_layer = CIN(cin_size)
        self.out_layer = Dense(1, activation=None)

    def call(self, inputs, training=None, mask=None):
        dense_inputs, sparse_inputs = inputs[:, :13], inputs[:, 13:]

        # linear
        linear_out = self.linear(inputs)

        emb = [self.embed_layers[i](sparse_inputs[:, i]) for i in range(sparse_inputs.shape[1])] # [n, None, k]
        emb = tf.transpose(tf.convert_to_tensor(emb), [1, 0, 2]) # [None, n, k]

        # CIN
        cin_out = self.cin_layer(emb)

        # dense
        emb = tf.reshape(emb, shape=(-1, emb.shape[1]*emb.shape[2]))
        emb = tf.concat([dense_inputs, emb], axis=1)
        dense_out = self.dense_layer(emb)

        output = self.out_layer(linear_out + cin_out + dense_out)
        return tf.nn.sigmoid(output)

In [7]:
'''
# Time   : 2021/1/3 11:08
# Author : junchaoli
# File   : utils.py
'''

import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

def sparseFeature(feat, feat_onehot_dim, embed_dim):
    return {'feat': feat, 'feat_onehot_dim': feat_onehot_dim, 'embed_dim': embed_dim}

def denseFeature(feat):
    return {'feat': feat}

def create_criteo_dataset(file_path, embed_dim=8, test_size=0.2):
    data = pd.read_csv(file_path)

    dense_features = ['I' + str(i) for i in range(1, 14)]
    sparse_features = ['C' + str(i) for i in range(1, 27)]

    #缺失值填充
    data[dense_features] = data[dense_features].fillna(0)
    data[sparse_features] = data[sparse_features].fillna('-1')

    #归一化
    data[dense_features] = MinMaxScaler().fit_transform(data[dense_features])
    #LabelEncoding编码
    for col in sparse_features:
        data[col] = LabelEncoder().fit_transform(data[col]).astype(int)

    feature_columns = [[denseFeature(feat) for feat in dense_features]] + \
           [[sparseFeature(feat, data[feat].nunique(), embed_dim) for feat in sparse_features]]

    X = data.drop(['label'], axis=1).values
    y = data['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    return feature_columns, (X_train, y_train), (X_test, y_test)

In [16]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import losses, optimizers
from sklearn.metrics import accuracy_score
%load_ext tensorboard

#create a callback
tf_call_back = tf.keras.callbacks.TensorBoard(log_dir="./logs")

if __name__ == '__main__':
    file = '../../代码/推荐算法-FM/data/train.txt'
    test_size = 0.2
    hidden_units = [256, 128, 64]
    dropout = 0.3
    cin_size = [128, 128]

    feature_columns, (X_train, y_train), (X_test, y_test) = create_criteo_dataset(file, test_size=test_size)

    model = xDeepFM(feature_columns, cin_size, hidden_units, dropout=dropout)
    optimizer = optimizers.SGD(0.01)

    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

    # 训练方式一
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(train_dataset, epochs=100, callbacks=[tf_call_back])
    logloss, auc = model.evaluate(X_test, y_test)
    print('logloss {}\nAUC {}'.format(round(logloss,2), round(auc,2)))
    model.summary()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/100
50/50 [==============================] - 2s 12ms/step - loss: 30.3356 - accuracy: 0.6329
Epoch 2/100
50/50 [==============================] - 1s 12ms/step - loss: 11.7153 - accuracy: 0.5291
Epoch 3/100
50/50 [==============================] - 1s 11ms/step - loss: 10.5322 - accuracy: 0.5278
Epoch 4/100
50/50 [==============================] - 1s 12ms/step - loss: 8.6347 - accuracy: 0.5616
Epoch 5/100
50/50 [==============================] - 1s 12ms/step - loss: 7.9350 - accuracy: 0.5553
Epoch 6/100
50/50 [==============================] - 1s 12ms/step - loss: 7.4221 - accuracy: 0.5816
Epoch 7/100
50/50 [==============================] - 1s 12ms/step - loss: 6.2215 - accuracy: 0.6041
Epoch 8/100
50/50 [==============================] - 1s 12ms/step - loss: 5.2569 - accuracy: 0.6373
Epoch 9/100
50/50 [==============================] - 1s 12ms/step - loss: 4.9626 - accuracy: 0.6660
Epoch 1

50/50 [==============================] - 1s 12ms/step - loss: 0.0573 - accuracy: 0.9819
Epoch 82/100
50/50 [==============================] - 1s 11ms/step - loss: 0.0228 - accuracy: 0.9987
Epoch 83/100
50/50 [==============================] - 1s 11ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 84/100
50/50 [==============================] - 1s 11ms/step - loss: 0.2258 - accuracy: 0.9781
Epoch 85/100
50/50 [==============================] - 1s 11ms/step - loss: 0.0047 - accuracy: 0.9994
Epoch 86/100
50/50 [==============================] - 1s 11ms/step - loss: 0.0167 - accuracy: 0.9987
Epoch 87/100
50/50 [==============================] - 1s 11ms/step - loss: 0.2134 - accuracy: 0.9787
Epoch 88/100
50/50 [==============================] - 1s 12ms/step - loss: 0.0047 - accuracy: 0.9994
Epoch 89/100
50/50 [==============================] - 1s 11ms/step - loss: 0.0076 - accuracy: 0.9994
Epoch 90/100
50/50 [==============================] - 1s 11ms/step - loss: 0.0085 - accuracy: 0.9994
Epo

In [17]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 69756), started 0:21:21 ago. (Use '!kill 69756' to kill it.)